<a href="https://colab.research.google.com/github/Elwing-Chou/ml0223/blob/main/transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile
path = "drive/MyDrive/additional/train.zip"
f = zipfile.ZipFile(path)
f.extractall()

In [ ]:
import glob
from PIL import Image
fs = glob.glob("train/*")
Image.open(fs[5000]).resize((224, 224)).convert("RGB")

In [ ]:
import pandas as pd
dog = glob.glob("train/dog.*")
cat = glob.glob("train/cat.*")
datas = pd.DataFrame({
    "path":dog + cat,
    "ans":[0] * len(dog) + [1] * len(cat)
})
datas

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
vgg = VGG16(include_top=False, input_shape=(224, 224, 3))
vgg.summary()

In [ ]:
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
vgg_layers = vgg.layers
for l in vgg_layers:
    l.trainable = False
clf_layers = [
    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.25),
    Dense(2, activation="softmax")
]
model = Sequential(vgg_layers + clf_layers)
model.summary()

In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
model.compile(loss=SparseCategoricalCrossentropy(), 
       optimizer="adam",
       metrics=["accuracy"])

In [ ]:
# demo pre: 你的預處理也要抄當初的預處理
# https://github.com/keras-team/keras-applications/blob/master/keras_applications/imagenet_utils.py
import numpy as np
from tensorflow.keras.applications.vgg16 import preprocess_input
img = Image.open(fs[5000]).resize((224, 224)).convert("RGB")
img_np = np.array(img)
preprocess_input(img_np)

In [ ]:
x, y = np.array(datas["path"]), np.array(datas["ans"])

In [ ]:
def get_data(x, y, batch=20):
    idx = np.random.randint(0, len(x), size=batch)
    y_batch = y[idx]
    x_batch, x_batch_ori = [], []
    for p in x[idx]:
        img = Image.open(p).resize((224, 224)).convert("RGB")
        img_np = np.array(img) 
        img_np_pre = preprocess_input(img_np)
        x_batch.append(img_np_pre)
        x_batch_ori.append(img_np)
    return (np.array(x_batch), np.array(y_batch), np.array(x_batch_ori))
x_batch, y_batch, x_batch_ori = get_data(x, y)
print(x_batch.shape)
print(y_batch.shape)
print(x_batch_ori.shape)

In [ ]:
for i in range(20):
    print("-" * 15, i, "-" * 15)
    x_batch, y_batch, x_batch_ori = get_data(x, y)
    result = model.train_on_batch(x_batch, y_batch)
    print("[Train]:", result)
    x_batch, y_batch, x_batch_ori = get_data(x, y)
    result = model.test_on_batch(x_batch, y_batch)
    print("[Validate]:", result)